In [ ]:
pip install pandas plotly


In [ ]:
pip install prophet

In [1]:
'''пример файла
import pandas as pd

# Создание данных о продажах
data = {
    "Region": ["North", "South", "East", "West", "North", "South", "East", "West", "North", "South", "East", "West"],
    "Product": ["A", "B", "C", "D", "A", "B", "C", "D", "A", "B", "C", "D"],
    "Sales": [15000, 20000, 18000, 22000, 16000, 21000, 19000, 23000, 17000, 22000, 20000, 24000],
    "Profit": [5000, 7000, 6000, 8000, 5500, 7500, 6500, 8500, 5700, 7700, 6700, 8700],
    "Month": [
        "2023-01-01",
        "2023-01-01",
        "2023-01-01",
        "2023-01-01",
        "2023-02-01",
        "2023-02-01",
        "2023-02-01",
        "2023-02-01",
        "2023-03-01",
        "2023-03-01",
        "2023-03-01",
        "2023-03-01"
    ]
}

# Создание DataFrame
df = pd.DataFrame(data)

# Сохранение DataFrame в CSV файл
file_path = 'sales_data.csv'
df.to_csv(file_path, index=False)

print(f"Файл '{file_path}' успешно создан.")'''


Файл 'sales_data.csv' успешно создан.


In [ ]:
'''Описание функционала:
Суммарные продажи по регионам (Bar chart):
Сравнение общего объема продаж между регионами.

Динамика продаж по месяцам (Line chart):
График изменения продаж во времени для каждого региона.

Доли продаж по регионам (Pie chart):
Круговая диаграмма, показывающая вклад каждого региона в общий объем продаж.'''
import pandas as pd
import plotly.express as px

# 1. Загрузка данных
file_path = "sales_data.csv"  
try:
    data = pd.read_csv(file_path)
    data['Month'] = pd.to_datetime(data['Month'])
except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")
    exit()

# 2. Проверка структуры данных
required_columns = {"Region", "Sales", "Month"}
if not required_columns.issubset(data.columns):
    print(f"Ошибка: В файле отсутствуют необходимые колонки: {required_columns}")
    exit()

# 3. Агрегация данных по регионам
region_sales = data.groupby(['Region'], as_index=False).agg({'Sales': 'sum'})

# 4. Визуализация продаж по регионам (суммарно)
fig1 = px.bar(region_sales, 
              x='Region', 
              y='Sales', 
              title="Суммарные продажи по регионам",
              labels={'Sales': 'Продажи', 'Region': 'Регион'},
              text='Sales')
fig1.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig1.show()

# 5. Визуализация динамики продаж по месяцам
fig2 = px.line(data, 
               x='Month', 
               y='Sales', 
               color='Region',
               title="Динамика продаж по месяцам",
               labels={'Sales': 'Продажи', 'Month': 'Месяц', 'Region': 'Регион'})
fig2.show()

# 6. Визуализация долей продаж по регионам (Pie chart)
fig3 = px.pie(region_sales, 
              names='Region', 
              values='Sales', 
              title="Распределение продаж по регионам")
fig3.show()


In [ ]:
'''Расширенный код с добавлением следующих функций:

Мультипродуктный прогноз:
Для всех указанных продуктов строится прогноз и сохраняется отдельный файл для каждого.

Сравнительный график:
Визуализация исторической популярности всех продуктов на одном графике для сравнения.

Учет сезонности и трендов:
Добавлен параметр для настройки сезонности в Prophet.'''
import pandas as pd
import plotly.express as px
from prophet import Prophet
import os


# 1. Загрузка данных
file_path = "sales_data.csv"  
try:
    data = pd.read_csv(file_path)
    data['Month'] = pd.to_datetime(data['Month'])
except Exception as e:
    print(f"Ошибка при загрузке данных: {e}")
    exit()

# 2. Проверка структуры данных
required_columns = {"Product", "Region", "Sales", "Month"}
if not required_columns.issubset(data.columns):
    print(f"Ошибка: В файле отсутствуют необходимые колонки: {required_columns}")
    exit()

# 3. Агрегация данных по регионам
region_sales = data.groupby(['Region'], as_index=False).agg({'Sales': 'sum'})

# 4. Визуализация продаж по регионам (суммарно)
fig1 = px.bar(region_sales, 
              x='Region', 
              y='Sales', 
              title="Суммарные продажи по регионам",
              labels={'Sales': 'Продажи', 'Region': 'Регион'},
              text='Sales')
fig1.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig1.show()

# 5. Визуализация динамики продаж по месяцам
fig2 = px.line(data, 
               x='Month', 
               y='Sales', 
               color='Region',
               title="Динамика продаж по месяцам",
               labels={'Sales': 'Продажи', 'Month': 'Месяц', 'Region': 'Регион'})
fig2.show()

# 6. Визуализация долей продаж по регионам (Pie chart)
fig3 = px.pie(region_sales, 
              names='Region', 
              values='Sales', 
              title="Распределение продаж по регионам")
fig3.show()

# 7. Мультипродуктный прогноз
output_dir = "forecasts"
os.makedirs(output_dir, exist_ok=True)

products = data['Product'].unique()
for product in products:
    product_data = data[data['Product'] == product]
    product_data = product_data.groupby('Month', as_index=False).agg({'Sales': 'sum'})
    product_data = product_data.rename(columns={'Month': 'ds', 'Sales': 'y'})

    # Создание модели Prophet
    model = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    model.fit(product_data)
    
    # Прогноз на 12 месяцев вперед
    future = model.make_future_dataframe(periods=12, freq='M')
    forecast = model.predict(future)
    
    # Сохранение прогноза
    forecast_file = os.path.join(output_dir, f"{product}_forecast.csv")
    forecast.to_csv(forecast_file, index=False)
    print(f"Прогноз для продукта '{product}' сохранен в файл: {forecast_file}")

    # Визуализация прогноза
    fig_forecast = model.plot(forecast)
    fig_forecast.show()

# 8. Сравнительный график популярности продуктов
comparison_data = data.groupby(['Month', 'Product'], as_index=False).agg({'Sales': 'sum'})
fig4 = px.line(comparison_data,
               x='Month', 
               y='Sales', 
               color='Product',
               title="Сравнительная популярность продуктов",
               labels={'Sales': 'Продажи', 'Month': 'Месяц', 'Product': 'Продукт'})
fig4.show()

# 9. Учет сезонности и трендов
# Пример с настройкой сезонности для одного продукта
sample_product = products[0]
sample_data = data[data['Product'] == sample_product]
sample_data = sample_data.groupby('Month', as_index=False).agg({'Sales': 'sum'})
sample_data = sample_data.rename(columns={'Month': 'ds', 'Sales': 'y'})

# Создание модели с учетом пользовательской настройки сезонности
model_custom = Prophet(yearly_seasonality=10)  # Повышенная детализация сезонности
model_custom.fit(sample_data)

# Прогноз и визуализация
future_custom = model_custom.make_future_dataframe(periods=12, freq='M')
forecast_custom = model_custom.predict(future_custom)
fig_custom = model_custom.plot(forecast_custom)
fig_custom.show()
